In [11]:
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
import numpy as np
from admm.agents import GlobalConsensus, EventGlobalConsensus, EventGlobalConsensusTorch
from admm.models import NN, Dummy
from admm.utils import add_params, average_params, sum_params
import torch
from tqdm import tqdm

%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Experimenting with SGD and adding parameters

In [12]:
def get_copy(params):
    copy = [torch.zeros(param.shape).copy_(param) for param in params]
    return copy

In [13]:
model1 = NN(2,2,1)

copy = get_copy(model1.parameters())

for param in copy:
    param.data = 2*torch.ones(param.shape)
for param in copy:
    print(param)

print('\n----- model 1 -----')
print('before training')
for param in model1.parameters():
    print(param)

# experiment minimizing l(x) = |x - 2|_2
opt = torch.optim.Adam(model1.parameters(), lr=0.01)
loss = torch.Tensor([np.inf])
prev_loss = torch.Tensor([0])
while np.abs(prev_loss.item() - loss.item()) >= 1e-4:
    prev_loss = loss
    opt.zero_grad()
    loss = 0
    for param, copied in zip(model1.parameters(), copy):
        loss += torch.norm(param-copied.data)
    loss.backward()
    opt.step()

print('\nafter training')
for param in model1.parameters():
    print(param)
print('\ncopied')
for param in copy:
    print(param)

tensor([[2., 2.],
        [2., 2.]], grad_fn=<CopyBackwards>)
tensor([2., 2.], grad_fn=<CopyBackwards>)
tensor([[2., 2.]], grad_fn=<CopyBackwards>)
tensor([2.], grad_fn=<CopyBackwards>)

----- model 1 -----
before training
Parameter containing:
tensor([[-0.2199,  0.3507],
        [-0.3482, -0.3243]], requires_grad=True)
Parameter containing:
tensor([-0.1722,  0.6059], requires_grad=True)
Parameter containing:
tensor([[ 0.2196, -0.6402]], requires_grad=True)
Parameter containing:
tensor([-0.3818], requires_grad=True)

after training
Parameter containing:
tensor([[1.9997, 1.9996],
        [1.9980, 1.9994]], requires_grad=True)
Parameter containing:
tensor([2.0000, 2.0018], requires_grad=True)
Parameter containing:
tensor([[1.9997, 1.9978]], requires_grad=True)
Parameter containing:
tensor([1.9972], requires_grad=True)

copied
tensor([[2., 2.],
        [2., 2.]], grad_fn=<CopyBackwards>)
tensor([2., 2.], grad_fn=<CopyBackwards>)
tensor([[2., 2.]], grad_fn=<CopyBackwards>)
tensor([2.], gra

### Event-Based ADMM with Torch

In [14]:
# Initial lambdas must sum to 0!
delta = 0.01
rho = 0.01
N = 4

agents = [
    EventGlobalConsensusTorch(
        N=N, 
        rho=rho, 
        model=NN(2,3,1),
        delta=delta
    ) 
    for _ in range(N)
]

for agent in agents:
    agent.primal_avg = average_params([agent.model.parameters() for agent in agents])
    for param in agent.primal_avg:
        print(param)

for agent in agents:
    for param in agent.primal_avg:
        print(param)

print('\nbefore training')
for agent in agents:
    for param in agent.model.parameters(): print(param)

tensor([[-0.0381,  0.0534],
        [ 0.2208,  0.0162],
        [-0.0824, -0.2614]], grad_fn=<DivBackward0>)
tensor([ 0.0612,  0.2636, -0.4839], grad_fn=<DivBackward0>)
tensor([[ 0.1654,  0.2116, -0.2020]], grad_fn=<DivBackward0>)
tensor([-0.0667], grad_fn=<DivBackward0>)
tensor([[-0.0381,  0.0534],
        [ 0.2208,  0.0162],
        [-0.0824, -0.2614]], grad_fn=<DivBackward0>)
tensor([ 0.0612,  0.2636, -0.4839], grad_fn=<DivBackward0>)
tensor([[ 0.1654,  0.2116, -0.2020]], grad_fn=<DivBackward0>)
tensor([-0.0667], grad_fn=<DivBackward0>)
tensor([[-0.0381,  0.0534],
        [ 0.2208,  0.0162],
        [-0.0824, -0.2614]], grad_fn=<DivBackward0>)
tensor([ 0.0612,  0.2636, -0.4839], grad_fn=<DivBackward0>)
tensor([[ 0.1654,  0.2116, -0.2020]], grad_fn=<DivBackward0>)
tensor([-0.0667], grad_fn=<DivBackward0>)
tensor([[-0.0381,  0.0534],
        [ 0.2208,  0.0162],
        [-0.0824, -0.2614]], grad_fn=<DivBackward0>)
tensor([ 0.0612,  0.2636, -0.4839], grad_fn=<DivBackward0>)
tensor([[ 0.

### Run simulation

In [15]:
comm = 0

t_max = 200
for t in tqdm(range(t_max)):
    
    # Primal Update
    for agent in agents:
        agent.primal_update()

    # Residual update in the case of communication
    C = []
    for agent in agents:
        if agent.broadcast: 
            comm += 1
            C.append(agent.residual)
    if C:
        # If communicaiton set isn't empty
        residuals = [x for x in sum_params(C)]
        for agent in agents:
            add_params(agent.primal_avg, residuals)

    # Dual update
    for agent in agents:
        agent.dual_update()

load = comm/(t_max*len(agents))
print(f'Communication load = {load}')

print('\nParams')
for agent in agents:
    for param in agent.model.parameters():
        print(param)

print('\nAverage')
for agent in agents:
    for param in agent.primal_avg:
        print(param)

100%|██████████| 200/200 [00:35<00:00,  5.69it/s]

Communication load = 0.38375

Params
Parameter containing:
tensor([[10.0002, 10.0004],
        [10.0003,  9.9995],
        [10.0003,  9.9990]], requires_grad=True)
Parameter containing:
tensor([ 9.9995,  9.9996, 10.0011], requires_grad=True)
Parameter containing:
tensor([[10.0000, 10.0000, 10.0001]], requires_grad=True)
Parameter containing:
tensor([9.9991], requires_grad=True)
Parameter containing:
tensor([[10.0004,  9.9995],
        [10.0006, 10.0006],
        [ 9.9995,  9.9997]], requires_grad=True)
Parameter containing:
tensor([ 9.9995,  9.9995, 10.0006], requires_grad=True)
Parameter containing:
tensor([[10.0004,  9.9995,  9.9997]], requires_grad=True)
Parameter containing:
tensor([10.0005], requires_grad=True)
Parameter containing:
tensor([[10.0003,  9.9996],
        [10.0002,  9.9994],
        [ 9.9994,  9.9999]], requires_grad=True)
Parameter containing:
tensor([ 9.9991,  9.9997, 10.0007], requires_grad=True)
Parameter containing:
tensor([[10.0008, 10.0001,  9.9997]], requires_